In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
from functools import reduce
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.set_option('display.max_columns', 1000)

In [3]:
# Defining directories
dir_path = Path.cwd().parent
data_folder = dir_path.joinpath("data/processed")

#test frames
s1_path = data_folder.joinpath("sent1_test_season_medians_widened.parquet")
s2_path = data_folder.joinpath("sent2_test_season_medians_widened.parquet")
cov_path = data_folder.joinpath("cov_final.parquet")


valid_df_path = dir_path.joinpath("data/valid_df.csv")

index_cols = ["long", "lat", "species_names"]

In [4]:
s1 = pd.read_parquet(s1_path).drop('year', axis=1)
s1

,long,lat,species_names,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter
0,75.66669580000007,32.25107380000003,Bambusoideae spp.,22.329961,23.283796,20.855999,18.947904,22.922561,23.829795,20.844843,21.205739,1.137842,1.181994,1.104959,1.173773,0.879803,0.846028,0.905011,0.851953,-0.064192,-0.083407,-0.049863,-0.079941,-2.966616,-3.714562,-2.211696,-3.467875,44.128804,44.882130,44.355896,42.368646,-0.258039,-0.335966,-0.199947,-0.321819,-0.127756,-0.165662,-0.099478,-0.158866,-137.101837,-163.194938,-98.101772,-150.438419
1,75.66676390000003,32.25073910000003,Bambusoideae spp.,23.710679,23.786145,19.777236,19.955244,20.844843,19.280017,16.127839,17.322448,1.275488,1.113172,1.206650,1.142118,0.784014,0.898350,0.828741,0.875566,-0.121068,-0.053551,-0.093649,-0.066345,-4.383192,-1.957570,-4.132998,-2.975134,37.096939,36.602465,42.537256,43.741389,-0.491474,-0.214822,-0.377909,-0.266552,-0.238637,-0.106794,-0.185669,-0.132108,-170.728100,-71.635497,-182.401575,-133.415633
2,75.66686810000004,32.25026770000005,Bambusoideae spp.,24.082400,23.820170,21.139434,21.449505,22.718416,23.069685,20.469209,20.882193,1.080752,1.105246,1.140018,1.125443,0.925291,0.905036,0.877179,0.888539,-0.038806,-0.049917,-0.065428,-0.059020,-1.727975,-2.187492,-2.959898,-2.626042,42.846788,43.951877,45.243961,45.405797,-0.155461,-0.200209,-0.262839,-0.236903,-0.077496,-0.099564,-0.130299,-0.117629,-76.977260,-95.889959,-133.902116,-116.844349
3,75.66719240000003,32.249825100000066,Bambusoideae spp.,21.702617,23.463530,20.713199,20.934217,23.260264,23.891661,22.001567,20.290231,1.059116,1.105724,1.089245,1.104644,0.946510,0.904391,0.918067,0.905271,-0.028077,-0.050206,-0.042716,-0.049720,-1.251883,-2.286745,-1.760913,-2.228705,45.268645,42.717487,42.180100,44.362581,-0.112606,-0.201333,-0.171177,-0.199373,-0.056007,-0.100159,-0.085277,-0.099195,-55.821320,-104.322354,-72.590927,-99.906539
4,75.66729090000007,32.24962370000003,Bambusoideae spp.,25.352941,23.563249,22.088770,21.461280,22.510769,23.018103,21.205615,20.457983,1.126578,1.128651,1.099359,1.116795,0.888100,0.888449,0.913310,0.895420,-0.059386,-0.059713,-0.046272,-0.055175,-2.683331,-2.560120,-2.144307,-2.419482,43.709939,43.476086,46.636632,45.402293,-0.238479,-0.240201,-0.186049,-0.221375,-0.118311,-0.118761,-0.092069,-0.110016,-119.092017,-109.853732,-99.370167,-105.925533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148902,78.12497830000007,31.25865170000003,Rhododendron arboreum,18.888946,18.692317,16.334685,17.132556,23.207470,22.411509,21.139434,20.827854,1.086950,1.097356,1.088839,1.094687,0.920094,0.911281,0.918410,0.913667,-0.041639,-0.046419,-0.042530,-0.045157,-1.848726,-2.027719,-1.504102,-1.592134,43.413158,42.134957,35.298377,36.018095,-0.166856,-0.186075,-0.170429,-0.181020,-0.083127,-0.092637,-0.084907,-0.090117,-82.164870,-88.577731,-59.254315,-55.170010
148903,78.12522000000007,31.258880000000033,Rhododendron arboreum,17.708520,17.923917,16.627578,16.213949,22.671717,23.344351,20.148947,19.943686,1.078354,1.169498,1.112976,1.075553,0.927339,0.855068,0.898492,0.929755,-0.037700,-0.078128,-0.053468,-0.036401,-1.570466,-3.594330,-1.948022,-1.223179,43.256798,42.017216,33.823773,34.292983,-0.151015,-0.314430,-0.214484,-0.145798,-0.075293,-0.155308,-0.106631,-0.072706,-65.420747,-165.360064,-70.973345,-43.825039
148904,

In [5]:
s2 = pd.read_parquet(s2_path).drop('year', axis=1)
s2

,long,lat,species_names,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter
0,75.66669580000007,32.25107380000003,Bambusoideae spp.,0.29080,0.25830,0.14230,0.15780,0.22880,0.19190,0.33480,0.34770,0.16720,0.15900,0.1664,0.17030,0.285961,0.372410,0.154575,0.255719,0.405837,0.375668,2.065170,1.906218,0.886545,0.934029,0.209016,0.272798,0.254991,0.354750,0.203063,0.263581,0.154652,0.239437,0.155412,0.285022,0.253745,0.328322,-0.765746,-0.752987,15.826980,20.746814,0.775898,1.000841,37.068751,28.242038,0.195003,0.228759,0.285961,0.372410
1,75.66676390000003,32.25073910000003,Bambusoideae spp.,0.31710,0.27550,0.13660,0.15180,0.24580,0.19890,0.34540,0.33450,0.15810,0.15340,0.1648,0.16380,0.345016,0.372159,0.223668,0.255426,0.421399,0.375694,2.035980,1.912466,0.919247,0.933895,0.253276,0.283844,0.221278,0.337335,0.245149,0.273260,0.204307,0.232773,0.133574,0.262741,0.305154,0.336195,-0.755262,-0.766043,19.636599,15.742295,0.799167,0.971819,29.353192,15.439635,0.218914,0.226362,0.345016,0.372159
2,75.66686810000004,32.25026770000005,Bambusoideae spp.,0.32890,0.28610,0.14420,0.15360,0.25470,0.20680,0.33860,0.37100,0.16220,0.15900,0.1720,0.17120,0.317657,0.400000,0.191659,0.288000,0.388559,0.414411,1.992846,2.012628,0.904244,0.948683,0.219832,0.302408,0.221538,0.348604,0.213420,0.292518,0.181254,0.267172,0.120755,0.282627,0.271958,0.356406,-0.761988,-0.729942,16.993831,34.403773,0.767839,1.027362,57.208149,23.117523,0.201435,0.252455,0.317657,0.400000
3,75.66719240000003,32.249825100000066,Bambusoideae spp.,0.32510,0.27620,0.14340,0.15570,0.25590,0.19410,0.32540,0.34350,0.15900,0.15620,0.1667,0.17140,0.305036,0.406310,0.176892,0.295382,0.364361,0.382651,1.984213,1.934911,0.897238,0.952003,0.206904,0.287520,0.214636,0.348844,0.198928,0.276591,0.177901,0.248658,0.112903,0.277902,0.261741,0.352054,-0.775044,-0.757141,13.576633,15.099199,0.752091,0.996998,31.677736,14.018858,0.188160,0.226670,0.305036,0.406310
4,75.66729090000007,32.24962370000003,Bambusoideae spp.,0.32770,0.27370,0.13640,0.15710,0.25180,0.19400,0.34350,0.33040,0.15520,0.14800,0.1603,0.16480,0.349971,0.426893,0.229466,0.319465,0.423539,0.424003,2.084338,2.123584,0.921938,0.962753,0.255823,0.298361,0.202223,0.341800,0.247436,0.286588,0.210158,0.255555,0.112010,0.266512,0.308859,0.367321,-0.757141,-0.770098,18.811617,14.103185,0.781214,1.019590,29.773170,11.618175,0.219119,0.228862,0.349971,0.426893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148902,78.12497830000007,31.25865170000003,Rhododendron arboreum,0.21160,0.17545,0.16735,0.12860,0.17215,0.14170,0.25400,0.20655,0.15155,0.14475,0.1650,0.13740,0.237574,0.165224,0.097962,0.013313,0.194441,0.223169,1.435463,1.551780,0.855441,0.815613,0.156266,0.092444,0.302340,0.305836,0.149567,0.087003,0.144799,0.073740,0.196189,0.179411,0.200646,0.131142,-0.845664,-0.892596,5.744313,5.542639,0.775235,0.708139,9.573166,7.617956,0.140034,0.093198,0.237574,0.165224
148903,78.12522000000007,31.258880000000033,Rhododendron arboreum,0.22900,0.17825,0.17680,0.12045,0.18470,0.14705,0.29230,0.19560,0.15390,0.15065,0.1825,0.14280,0.225503,0.119334,0.083838,-0.040380,0.198402,0.228580,1.487546,1.417172,0.849130,0.786970,0.163073,0.065693,0.320799,0.277850,0.157876,0.061775,0.146941,0.052782,0.228374,0.141676,0.198501,0.094295,-0.807782,-0.903427,5.810397,5.304682,0.794209,0.643412,-7.820830,7.795563,0.159572,0.078161,0.225503,0.1193

In [6]:
cov_df = pd.read_parquet(cov_path)
cov_df

,long,lat,species_names,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,75.66669580000007,32.25107380000003,Bambusoideae spp.,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275
1,75.66676390000003,32.25073910000003,Bambusoideae spp.,26.49250,350.0,13.5475,4.685,6.29375,10.147419,22.47375,8.6365,10.74375,1.008625,254.256454,26.03375,7.0275
2,75.66686810000004,32.25026770000005,Bambusoideae spp.,26.49250,349.0,13.5475,4.685,6.29375,8.110854,22.47375,8.6365,10.74375,1.008625,250.076767,26.03375,7.0275
3,75.66719240000003,32.249825100000066,Bambusoideae spp.,26.49250,351.0,13.5475,4.685,6.29375,9.826173,22.47375,8.6365,10.74375,1.008625,242.139969,26.03375,7.0275
4,75.66729090000007,32.24962370000003,Bambusoideae spp.,26.49250,348.0,13.5475,4.685,6.29375,6.125500,22.47375,8.6365,10.74375,1.008625,243.095016,26.03375,7.0275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148902,78.12497830000007,31.25865170000003,Rhododendron arboreum,36.73625,2309.0,19.9450,4.570,30.67875,54.879799,16.27250,11.7515,14.13875,0.968750,325.060272,21.98625,12.1675
148903,78.12522000000007,31.258880000000033,Rhododendron arboreum,36.73625,2292.0,19.9450,4.570,30.67875,58.185696,16.27250,11.7515,14.13875,0.968750,319.749359,21.98625,12.1675
148904,78.12614830000007,31.258660000000077,Rhododendron arboreum,36.73625,2361.0,19.9450,4.570,30.06625,30.714142,16.27250,11.7515,14.13875,0.966375,292.424896,21.98625,12.1675
148905,78.12618260000005,31.25882800000005,Rhododendron arboreum,36.73625,2350.0,19.9450,4.570,30.06625,43.765118,16.27250,11.7515,14.13875,0.966375,348.596802,21.98625,12.1675


In [7]:
merged_df = reduce(lambda left, right: pd.merge(left, right,  on=index_cols, how='outer', validate="1:1"), [s1, s2, cov_df])
merged_df


,long,lat,species_names,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,75.66669580000007,32.25107380000003,Bambusoideae spp.,22.329961,23.283796,20.855999,18.947904,22.922561,23.829795,20.844843,21.205739,1.137842,1.181994,1.104959,1.173773,0.879803,0.846028,0.905011,0.851953,-0.064192,-0.083407,-0.049863,-0.079941,-2.966616,-3.714562,-2.211696,-3.467875,44.128804,44.882130,44.355896,42.368646,-0.258039,-0.335966,-0.199947,-0.321819,-0.127756,-0.165662,-0.099478,-0.158866,-137.101837,-163.194938,-98.101772,-150.438419,0.29080,0.25830,0.14230,0.15780,0.22880,0.19190,0.33480,0.34770,0.16720,0.15900,0.1664,0.17030,0.285961,0.372410,0.154575,0.255719,0.405837,0.375668,2.065170,1.906218,0.886545,0.934029,0.209016,0.272798,0.254991,0.354750,0.203063,0.263581,0.154652,0.239437,0.155412,0.285022,0.253745,0.328322,-0.765746,-0.752987,15.826980,20.746814,0.775898,1.000841,37.068751,28.242038,0.195003,0.228759,0.285961,0.372410,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275
1,75.66676390000003,32.25073910000003,Bambusoideae spp.,23.710679,23.786145,19.777236,19.955244,20.844843,19.280017,16.127839,17.322448,1.275488,1.113172,1.206650,1.142118,0.784014,0.898350,0.828741,0.875566,-0.121068,-0.053551,-0.093649,-0.066345,-4.383192,-1.957570,-4.132998,-2.975134,37.096939,36.602465,42.537256,43.741389,-0.491474,-0.214822,-0.377909,-0.266552,-0.238637,-0.106794,-0.185669,-0.132108,-170.728100,-71.635497,-182.401575,-133.415633,0.31710,0.27550,0.13660,0.15180,0.24580,0.19890,0.34540,0.33450,0.15810,0.15340,0.1648,0.16380,0.345016,0.372159,0.223668,0.255426,0.421399,0.375694,2.035980,1.912466,0.919247,0.933895,0.253276,0.283844,0.221278,0.337335,0.245149,0.273260,0.204307,0.232773,0.133574,0.262741,0.305154,0.336195,-0.755262,-0.766043,19.636599,15.742295,0.799167,0.971819,29.353192,15.439635,0.218914,0.226362,0.345016,0.372159,26.49250,350.0,13.5475,4.685,6.29375,10.147419,22.47375,8.6365,10.74375,1.008625,254.256454,26.03375,7.0275
2,75.66686810000004,32.25026770000005,Bambusoideae spp.,24.082400,23.820170,21.139434,21.449505,22.718416,23.069685,20.469209,20.882193,1.080752,1.105246,1.140018,1.125443,0.925291,0.905036,0.877179,0.888539,-0.038806,-0.049917,-0.065428,-0.059020,-1.727975,-2.187492,-2.959898,-2.626042,42.846788,43.951877,45.243961,45.405797,-0.155461,-0.200209,-0.262839,-0.236903,-0.077496,-0.099564,-0.130299,-0.117629,-76.977260,-95.889959,-133.902116,-116.844349,0.32890,0.28610,0.14420,0.15360,0.25470,0.20680,0.33860,0.37100,0.16220,0.15900,0.1720,0.17120,0.317657,0.400000,0.191659,0.288000,0.388559,0.414411,1.992846,2.012628,0.904244,0.948683,0.219832,0.302408,0.221538,0.348604,0.213420,0.292518,0.181254,0.267172,0.120755,0.282627,0.271958,0.356406,-0.761988,-0.729942,16.993831,34.403773,0.767839,1.027362,57.208149,23.117

In [8]:
merged_df["species"] = pd.Categorical(merged_df['species_names']).codes
merged_df.drop('species_names', axis=1, inplace=True)
merged_df

,long,lat,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f,species
0,75.66669580000007,32.25107380000003,22.329961,23.283796,20.855999,18.947904,22.922561,23.829795,20.844843,21.205739,1.137842,1.181994,1.104959,1.173773,0.879803,0.846028,0.905011,0.851953,-0.064192,-0.083407,-0.049863,-0.079941,-2.966616,-3.714562,-2.211696,-3.467875,44.128804,44.882130,44.355896,42.368646,-0.258039,-0.335966,-0.199947,-0.321819,-0.127756,-0.165662,-0.099478,-0.158866,-137.101837,-163.194938,-98.101772,-150.438419,0.29080,0.25830,0.14230,0.15780,0.22880,0.19190,0.33480,0.34770,0.16720,0.15900,0.1664,0.17030,0.285961,0.372410,0.154575,0.255719,0.405837,0.375668,2.065170,1.906218,0.886545,0.934029,0.209016,0.272798,0.254991,0.354750,0.203063,0.263581,0.154652,0.239437,0.155412,0.285022,0.253745,0.328322,-0.765746,-0.752987,15.826980,20.746814,0.775898,1.000841,37.068751,28.242038,0.195003,0.228759,0.285961,0.372410,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275,1
1,75.66676390000003,32.25073910000003,23.710679,23.786145,19.777236,19.955244,20.844843,19.280017,16.127839,17.322448,1.275488,1.113172,1.206650,1.142118,0.784014,0.898350,0.828741,0.875566,-0.121068,-0.053551,-0.093649,-0.066345,-4.383192,-1.957570,-4.132998,-2.975134,37.096939,36.602465,42.537256,43.741389,-0.491474,-0.214822,-0.377909,-0.266552,-0.238637,-0.106794,-0.185669,-0.132108,-170.728100,-71.635497,-182.401575,-133.415633,0.31710,0.27550,0.13660,0.15180,0.24580,0.19890,0.34540,0.33450,0.15810,0.15340,0.1648,0.16380,0.345016,0.372159,0.223668,0.255426,0.421399,0.375694,2.035980,1.912466,0.919247,0.933895,0.253276,0.283844,0.221278,0.337335,0.245149,0.273260,0.204307,0.232773,0.133574,0.262741,0.305154,0.336195,-0.755262,-0.766043,19.636599,15.742295,0.799167,0.971819,29.353192,15.439635,0.218914,0.226362,0.345016,0.372159,26.49250,350.0,13.5475,4.685,6.29375,10.147419,22.47375,8.6365,10.74375,1.008625,254.256454,26.03375,7.0275,1
2,75.66686810000004,32.25026770000005,24.082400,23.820170,21.139434,21.449505,22.718416,23.069685,20.469209,20.882193,1.080752,1.105246,1.140018,1.125443,0.925291,0.905036,0.877179,0.888539,-0.038806,-0.049917,-0.065428,-0.059020,-1.727975,-2.187492,-2.959898,-2.626042,42.846788,43.951877,45.243961,45.405797,-0.155461,-0.200209,-0.262839,-0.236903,-0.077496,-0.099564,-0.130299,-0.117629,-76.977260,-95.889959,-133.902116,-116.844349,0.32890,0.28610,0.14420,0.15360,0.25470,0.20680,0.33860,0.37100,0.16220,0.15900,0.1720,0.17120,0.317657,0.400000,0.191659,0.288000,0.388559,0.414411,1.992846,2.012628,0.904244,0.948683,0.219832,0.302408,0.221538,0.348604,0.213420,0.292518,0.181254,0.267172,0.120755,0.282627,0.271958,0.356406,-0.761988,-0.729942,16.993831,34.403773,0.767839,1.027362,57.208149,23.117523,0.201435,0.252455,0.317657,0.400000,26.49250,349.0,1

### Normalizing

In [9]:
df_index_cols = merged_df[['long', 'lat', 'species']]
df_x = merged_df.drop(['long', 'lat', 'species'], axis=1)

df_x

,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,22.329961,23.283796,20.855999,18.947904,22.922561,23.829795,20.844843,21.205739,1.137842,1.181994,1.104959,1.173773,0.879803,0.846028,0.905011,0.851953,-0.064192,-0.083407,-0.049863,-0.079941,-2.966616,-3.714562,-2.211696,-3.467875,44.128804,44.882130,44.355896,42.368646,-0.258039,-0.335966,-0.199947,-0.321819,-0.127756,-0.165662,-0.099478,-0.158866,-137.101837,-163.194938,-98.101772,-150.438419,0.29080,0.25830,0.14230,0.15780,0.22880,0.19190,0.33480,0.34770,0.16720,0.15900,0.1664,0.17030,0.285961,0.372410,0.154575,0.255719,0.405837,0.375668,2.065170,1.906218,0.886545,0.934029,0.209016,0.272798,0.254991,0.354750,0.203063,0.263581,0.154652,0.239437,0.155412,0.285022,0.253745,0.328322,-0.765746,-0.752987,15.826980,20.746814,0.775898,1.000841,37.068751,28.242038,0.195003,0.228759,0.285961,0.372410,26.49250,352.0,13.5475,4.685,6.29375,13.422151,22.47375,8.6365,10.74375,1.008625,254.256500,26.03375,7.0275
1,23.710679,23.786145,19.777236,19.955244,20.844843,19.280017,16.127839,17.322448,1.275488,1.113172,1.206650,1.142118,0.784014,0.898350,0.828741,0.875566,-0.121068,-0.053551,-0.093649,-0.066345,-4.383192,-1.957570,-4.132998,-2.975134,37.096939,36.602465,42.537256,43.741389,-0.491474,-0.214822,-0.377909,-0.266552,-0.238637,-0.106794,-0.185669,-0.132108,-170.728100,-71.635497,-182.401575,-133.415633,0.31710,0.27550,0.13660,0.15180,0.24580,0.19890,0.34540,0.33450,0.15810,0.15340,0.1648,0.16380,0.345016,0.372159,0.223668,0.255426,0.421399,0.375694,2.035980,1.912466,0.919247,0.933895,0.253276,0.283844,0.221278,0.337335,0.245149,0.273260,0.204307,0.232773,0.133574,0.262741,0.305154,0.336195,-0.755262,-0.766043,19.636599,15.742295,0.799167,0.971819,29.353192,15.439635,0.218914,0.226362,0.345016,0.372159,26.49250,350.0,13.5475,4.685,6.29375,10.147419,22.47375,8.6365,10.74375,1.008625,254.256454,26.03375,7.0275
2,24.082400,23.820170,21.139434,21.449505,22.718416,23.069685,20.469209,20.882193,1.080752,1.105246,1.140018,1.125443,0.925291,0.905036,0.877179,0.888539,-0.038806,-0.049917,-0.065428,-0.059020,-1.727975,-2.187492,-2.959898,-2.626042,42.846788,43.951877,45.243961,45.405797,-0.155461,-0.200209,-0.262839,-0.236903,-0.077496,-0.099564,-0.130299,-0.117629,-76.977260,-95.889959,-133.902116,-116.844349,0.32890,0.28610,0.14420,0.15360,0.25470,0.20680,0.33860,0.37100,0.16220,0.15900,0.1720,0.17120,0.317657,0.400000,0.191659,0.288000,0.388559,0.414411,1.992846,2.012628,0.904244,0.948683,0.219832,0.302408,0.221538,0.348604,0.213420,0.292518,0.181254,0.267172,0.120755,0.282627,0.271958,0.356406,-0.761988,-0.729942,16.993831,34.403773,0.767839,1.027362,57.208149,23.117523,0.201435,0.252455,0.317657,0.400000,26.49250,349.0,13.5475,4.685,6.29375,8.110854,22.47375,8.6365,10.74375,1.008625,250.076767,26.03375,7.0275
3,21.702617,23.463530,20.713199,20.934

In [10]:
min_max_scaler = MinMaxScaler()

df_scaled = pd.DataFrame(min_max_scaler.fit_transform(df_x.values), columns=df_x.columns)
df_scaled

,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,0.371959,0.455084,0.464106,0.361325,0.405180,0.451742,0.366469,0.402716,0.298073,0.392722,0.178605,0.290332,0.580951,0.467565,0.706423,0.558771,0.636999,0.531595,0.761002,0.628136,0.661338,0.559623,0.795720,0.694748,0.371463,0.408478,0.422399,0.397511,0.652475,0.550446,0.779227,0.651864,0.621917,0.513281,0.743113,0.605089,0.683132,0.587161,0.810504,0.737186,0.239871,0.263245,0.055689,0.358340,0.202767,0.216582,0.190148,0.260483,0.069155,0.173550,0.051599,0.241115,0.565163,0.629457,0.565163,0.629457,0.665859,0.617379,0.376458,0.427621,0.637946,0.690570,0.418833,0.406632,0.494561,0.631163,0.436792,0.448846,0.321072,0.429589,0.447645,0.636487,0.523472,0.553611,0.190148,0.260483,1.0,0.838289,0.219814,0.312803,0.912242,0.239857,0.362237,0.268051,0.565163,0.629457,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
1,0.444319,0.481412,0.393130,0.428761,0.290817,0.188732,0.125808,0.206830,0.519064,0.293331,0.296285,0.252644,0.350285,0.577762,0.554034,0.604755,0.406360,0.638763,0.621801,0.671269,0.544355,0.685908,0.667243,0.726633,0.176957,0.180453,0.366992,0.440215,0.426917,0.654233,0.648817,0.692665,0.386717,0.623639,0.595578,0.650424,0.628188,0.718966,0.691784,0.759669,0.276806,0.294886,0.051611,0.334397,0.233110,0.234804,0.199277,0.245644,0.062194,0.159784,0.050401,0.222633,0.635598,0.629149,0.635598,0.629149,0.685235,0.617410,0.368640,0.429625,0.701577,0.690291,0.475753,0.419350,0.448435,0.608249,0.494539,0.461578,0.375791,0.420976,0.421660,0.613105,0.588312,0.563454,0.199277,0.245644,1.0,0.837711,0.234598,0.298550,0.912238,0.238194,0.402805,0.265229,0.635598,0.629149,0.19650,0.024244,0.319782,0.469907,0.040873,0.149554,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
2,0.463800,0.483195,0.482755,0.528795,0.393943,0.407802,0.347304,0.386396,0.206415,0.281885,0.219177,0.232790,0.690488,0.591844,0.650814,0.630019,0.739940,0.651806,0.711516,0.694508,0.763627,0.669382,0.745688,0.749223,0.336002,0.382859,0.449455,0.491992,0.751590,0.666752,0.733141,0.714553,0.728527,0.637191,0.690356,0.674954,0.781374,0.684050,0.760086,0.781557,0.293378,0.314386,0.057048,0.341580,0.248996,0.255369,0.193421,0.286678,0.065330,0.173550,0.055793,0.243674,0.602968,0.663464,0.602968,0.663464,0.644347,0.662047,0.357089,0.461752,0.672385,0.720997,0.432741,0.440725,0.448792,0.623077,0.451002,0.486909,0.350388,0.465432,0.406407,0.633974,0.546443,0.588722,0.193421,0.286678,1.0,0.839866,0.214693,0.325828,0.912253,0.239191,0.373150,0.295957,0.602968,0.663464,0.19650,0.023912,0.319782,0.469907,0.040873,0.119539,0.585997,0.088080,0.101551,0.636993,0.696864,0.576992,0.118323
3,0.339082,0.464504,0.454711,0.494299,0.423768,0.455318,0.425485,0.356535,0.171678,0.282575,0.160421,0.208028,0.741585,0.59

### Validation Data Frame

In [11]:
valid_df = pd.concat([df_index_cols, df_scaled], axis=1)
valid_df

,long,lat,species,vv_Descendingsummer,vv_Descendingwinter,vh_Descendingsummer,vh_Descendingwinter,vv_Ascendingsummer,vv_Ascendingwinter,vh_Ascendingsummer,vh_Ascendingwinter,VV_VH_Asummer,VV_VH_Awinter,VV_VH_Dsummer,VV_VH_Dwinter,VH_VV_Asummer,VH_VV_Awinter,VH_VV_Dsummer,VH_VV_Dwinter,SAR_NDVI_Asummer,SAR_NDVI_Awinter,SAR_NDVI_Dsummer,SAR_NDVI_Dwinter,DVI_Asummer,DVI_Awinter,DVI_Dsummer,DVI_Dwinter,SVI_Asummer,SVI_Awinter,SVI_Dsummer,SVI_Dwinter,RDVI_Asummer,RDVI_Awinter,RDVI_Dsummer,RDVI_Dwinter,NRDVI_Asummer,NRDVI_Awinter,NRDVI_Dsummer,NRDVI_Dwinter,SSDVI_Asummer,SSDVI_Awinter,SSDVI_Dsummer,SSDVI_Dwinter,B11summer,B11winter,B02summer,B02winter,B12summer,B12winter,B08summer,B08winter,B04summer,B04winter,B03summer,B03winter,NDVIsummer,NDVIwinter,ARVI2summer,ARVI2winter,BWDRVIsummer,BWDRVIwinter,CVIsummer,CVIwinter,CTVIsummer,CTVIwinter,EVI2summer,EVI2winter,GVMIsummer,GVMIwinter,MSVAIhypersummer,MSVAIhyperwinter,MTVI2summer,MTVI2winter,MNDVIsummer,MNDVIwinter,OSAVIsummer,OSAVIwinter,PVIsummer,PVIwinter,SARVIsummer,SARVIwinter,SLAVIsummer,SLAVIwinter,TSAVI2summer,TSAVI2winter,WDVIsummer,WDVIwinter,WDRVIsummer,WDRVIwinter,sand_f,elevation,cfvo_f,phh2o_f,soc_f,slope,clay_f,nitrogen_f,cec_f,bdod_f,aspect,silt_f,ocd_f
0,75.66669580000007,32.25107380000003,1,0.371959,0.455084,0.464106,0.361325,0.405180,0.451742,0.366469,0.402716,0.298073,0.392722,0.178605,0.290332,0.580951,0.467565,0.706423,0.558771,0.636999,0.531595,0.761002,0.628136,0.661338,0.559623,0.795720,0.694748,0.371463,0.408478,0.422399,0.397511,0.652475,0.550446,0.779227,0.651864,0.621917,0.513281,0.743113,0.605089,0.683132,0.587161,0.810504,0.737186,0.239871,0.263245,0.055689,0.358340,0.202767,0.216582,0.190148,0.260483,0.069155,0.173550,0.051599,0.241115,0.565163,0.629457,0.565163,0.629457,0.665859,0.617379,0.376458,0.427621,0.637946,0.690570,0.418833,0.406632,0.494561,0.631163,0.436792,0.448846,0.321072,0.429589,0.447645,0.636487,0.523472,0.553611,0.190148,0.260483,1.0,0.838289,0.219814,0.312803,0.912242,0.239857,0.362237,0.268051,0.565163,0.629457,0.19650,0.024909,0.319782,0.469907,0.040873,0.197817,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
1,75.66676390000003,32.25073910000003,1,0.444319,0.481412,0.393130,0.428761,0.290817,0.188732,0.125808,0.206830,0.519064,0.293331,0.296285,0.252644,0.350285,0.577762,0.554034,0.604755,0.406360,0.638763,0.621801,0.671269,0.544355,0.685908,0.667243,0.726633,0.176957,0.180453,0.366992,0.440215,0.426917,0.654233,0.648817,0.692665,0.386717,0.623639,0.595578,0.650424,0.628188,0.718966,0.691784,0.759669,0.276806,0.294886,0.051611,0.334397,0.233110,0.234804,0.199277,0.245644,0.062194,0.159784,0.050401,0.222633,0.635598,0.629149,0.635598,0.629149,0.685235,0.617410,0.368640,0.429625,0.701577,0.690291,0.475753,0.419350,0.448435,0.608249,0.494539,0.461578,0.375791,0.420976,0.421660,0.613105,0.588312,0.563454,0.199277,0.245644,1.0,0.837711,0.234598,0.298550,0.912238,0.238194,0.402805,0.265229,0.635598,0.629149,0.19650,0.024244,0.319782,0.469907,0.040873,0.149554,0.585997,0.088080,0.101551,0.636993,0.708511,0.576992,0.118323
2,75.66686810000004,32.25026770000005,1,0.463800,0.483195,0.482755,0.528795,0.393943,0.407802,0.347304,0.386396,0.206415,0.281885,0.219177,0.232790,0.690488,0.591844,0.650814,0.630019,0.739940,0.651806,0.711516,0.694508,0.763627,0.669382,0.745688,0.749223,0.336002,0.382859,0.449455,0.491992,0.751590,0.666752,0.733141,0.714553,0.728527,0.637191,0.690356,0.674954,0.781374,0.684050,0.760086,0.781557,0.293378,0.314386,0.057048,0.341580,0.248996,0.255369,0.193421,0.286678,0.065330,0.173550,0.055793,0.243674,0.602968,0.663464,0.602968,0.663464,0.644347,0.662047,0.357089,0.461752,0.672385,0.720997,0.432741,0.440725,0.448792,0.623077,0.451002,0.486909,0.350388,0.465432,0.406407,0.633974,0.546443,0.588722,0.193421,0.286678,1.0,0.839866,0.214693,0.325828,0.912253,0.239191,0.373150,0.295957,0.602968,0.663464,0.19650,0.023912,0.319782,0.469907,0.040873,0.119539,0.585997,0.088080,0.101551,0.636993,0.696864,0.576992,0

In [12]:
for col in valid_df.columns:
    print(pd.Series(valid_df[col].isna().value_counts()))

long
False    148907
Name: count, dtype: int64
lat
False    148907
Name: count, dtype: int64
species
False    148907
Name: count, dtype: int64
vv_Descendingsummer
False    148907
Name: count, dtype: int64
vv_Descendingwinter
False    148907
Name: count, dtype: int64
vh_Descendingsummer
False    148907
Name: count, dtype: int64
vh_Descendingwinter
False    148907
Name: count, dtype: int64
vv_Ascendingsummer
False    148907
Name: count, dtype: int64
vv_Ascendingwinter
False    148907
Name: count, dtype: int64
vh_Ascendingsummer
False    148907
Name: count, dtype: int64
vh_Ascendingwinter
False    148907
Name: count, dtype: int64
VV_VH_Asummer
False    148907
Name: count, dtype: int64
VV_VH_Awinter
False    148907
Name: count, dtype: int64
VV_VH_Dsummer
False    148907
Name: count, dtype: int64
VV_VH_Dwinter
False    148907
Name: count, dtype: int64
VH_VV_Asummer
False    148907
Name: count, dtype: int64
VH_VV_Awinter
False    148907
Name: count, dtype: int64
VH_VV_Dsummer
False    148907

In [13]:
valid_df.to_csv(valid_df_path, index=False)